In [2]:
import json
from pathlib import Path

import pandas as pd

In [3]:
from mhqa.dvc import load_experiments

filepaths = list(Path("../../tmp/frames-mini/").glob("*.json"))
experiments = [exp for fp in filepaths for exp in load_experiments(fp)]
print(f"{len(experiments)} experiments")
experiments[0]

8 experiments


{'commit': 'workspace',
 'id': 'workspace',
 'name': None,
 'params': {'train': {'dataset': {'path': 'bdsaglam/musique-sweep',
    'name': 'answerable',
    'split': 'train'},
   'optimizer': 'noop',
   'ensemble': 'no'},
  'evaluation': {'dataset': {'path': 'bdsaglam/musique-sweep',
    'name': 'answerable',
    'split': 'validation'}},
  'retrieve': {'model_type': 't5', 'top_k': 3},
  'qa': {'model': 'llama-3.3-70b',
   'temperature': 0.1,
   'technique': 'multihop-decompose'},
  'run': 1},
 'metrics': {}}

In [4]:
for exp in experiments:
    train_params = exp["params"]["train"]
    if train_params.get("ensemble") is None:
        train_params["ensemble"] = "no"

experiments[0]

{'commit': 'workspace',
 'id': 'workspace',
 'name': None,
 'params': {'train': {'dataset': {'path': 'bdsaglam/musique-sweep',
    'name': 'answerable',
    'split': 'train'},
   'optimizer': 'noop',
   'ensemble': 'no'},
  'evaluation': {'dataset': {'path': 'bdsaglam/musique-sweep',
    'name': 'answerable',
    'split': 'validation'}},
  'retrieve': {'model_type': 't5', 'top_k': 3},
  'qa': {'model': 'llama-3.3-70b',
   'temperature': 0.1,
   'technique': 'multihop-decompose'},
  'run': 1},
 'metrics': {}}

In [5]:
df = pd.json_normalize(experiments).drop(columns=["commit"])
df.drop(columns=[col for col in df.columns if "fuzzy" in col], inplace=True)
print(f"{len(df)} experiments before preprocessing")
df


8 experiments before preprocessing


,id,name,params.train.dataset.path,params.train.dataset.name,params.train.dataset.split,params.train.optimizer,params.train.ensemble,params.evaluation.dataset.path,params.evaluation.dataset.name,params.evaluation.dataset.split,params.retrieve.model_type,params.retrieve.top_k,params.qa.model,params.qa.temperature,params.qa.technique,params.run,metrics.exact_match,metrics.f1,metrics.0hops.exact_match,metrics.0hops.f1
0,workspace,None,bdsaglam/musique-sweep,answerable,train,noop,no,bdsaglam/musique-sweep,answerable,validation,t5,3,llama-3.3-70b,0.1,multihop-decompose,1,NaN,NaN,NaN,NaN
1,9e484d88c6fcac13b1105b1c943d5c5265efd988,chief-aces,bdsaglam/drop-musique-mini,default,train,miprov2-light,no,bdsaglam/drop-musique-mini,default,validation,golden,1,qwen-2.5-32b,0.0,cot,1,0.02,0.538309,0.02,0.538309
2,f596e8fc68c80fcd25418b35f5d495529d5b5735,wired-prob,bdsaglam/drop-musique-mini,default,train,noop,no,bdsaglam/drop-musique-mini,default,validation,golden,1,qwen-2.5-32b,0.0,cot,1,0.02,0.491522,0.02,0.491522
3,e9628a9a9bb5c8b8185e196048ac5398de34a0d3,worse-rasp,bdsaglam/drop-musique-mini,default,train,bfsrs-light,no,bdsaglam/drop-musique-mini,default,validation,golden,1,qwen-2.5-32b,0.0,cot,1,0.02,0.531318,0.02,0.531318
4,f21549ae4f971326fd434ca7ebf956262fc8797b,irate-matt,bdsaglam/drop-musique-mini,default,train,miprov2-light,no,bdsaglam/drop-musique-mini,default,validation,golden,1,qwen-2.5-32b,0.5,cot,1,0.01,0.517477,0.01,0.517477
5,21d3fddea9086cdbd0ce87082ce77ae4b1651a35,mazed-jato,bdsaglam/drop-musique-mini,default,train,noop,no,bdsaglam/drop-musique-mini,default,validation,golden,1,qwen-2.5-32b,0.5,cot,1,0.02,0.492792,0.02,0.492792
6,cba2044d10ad308d9c3112b5c735cc9d08100b96,urban-aria,bdsaglam/drop-musique-mini,default,train,noop,no,bdsaglam/drop-musique-mini,default,validation,golden,1,qwen-2.5-32b,0.5,cot,1,0.02,0.492792,0.02,0.492792
7,ea162084da1636fb65a219eca1647b56f51a4dd7,inter-pupa,bdsaglam/drop-musique-mini,default,train,bfsrs-light,no,bdsaglam/drop-musique-mini,default,validation,golden,1,qwen-2.5-32b,0.5,cot,1,0.01,0.519027,0.01,0.519027


In [6]:
mask = df["params.evaluation.dataset.path"] == "bdsaglam/drop-musique-mini"

df = df.loc[mask].copy()

param_cols = [col for col in df.columns if col.startswith("params.")]
metric_cols = [col for col in df.columns if col.startswith("metrics.")]
df.dropna(subset=param_cols + metric_cols, inplace=True, how="any")
df.drop_duplicates(subset=param_cols, inplace=True)

print(f"{len(df)} experiments after preprocessing")
df

6 experiments after preprocessing


,id,name,params.train.dataset.path,params.train.dataset.name,params.train.dataset.split,params.train.optimizer,params.train.ensemble,params.evaluation.dataset.path,params.evaluation.dataset.name,params.evaluation.dataset.split,params.retrieve.model_type,params.retrieve.top_k,params.qa.model,params.qa.temperature,params.qa.technique,params.run,metrics.exact_match,metrics.f1,metrics.0hops.exact_match,metrics.0hops.f1
1,9e484d88c6fcac13b1105b1c943d5c5265efd988,chief-aces,bdsaglam/drop-musique-mini,default,train,miprov2-light,no,bdsaglam/drop-musique-mini,default,validation,golden,1,qwen-2.5-32b,0.0,cot,1,0.02,0.538309,0.02,0.538309
2,f596e8fc68c80fcd25418b35f5d495529d5b5735,wired-prob,bdsaglam/drop-musique-mini,default,train,noop,no,bdsaglam/drop-musique-mini,default,validation,golden,1,qwen-2.5-32b,0.0,cot,1,0.02,0.491522,0.02,0.491522
3,e9628a9a9bb5c8b8185e196048ac5398de34a0d3,worse-rasp,bdsaglam/drop-musique-mini,default,train,bfsrs-light,no,bdsaglam/drop-musique-mini,default,validation,golden,1,qwen-2.5-32b,0.0,cot,1,0.02,0.531318,0.02,0.531318
4,f21549ae4f971326fd434ca7ebf956262fc8797b,irate-matt,bdsaglam/drop-musique-mini,default,train,miprov2-light,no,bdsaglam/drop-musique-mini,default,validation,golden,1,qwen-2.5-32b,0.5,cot,1,0.01,0.517477,0.01,0.517477
5,21d3fddea9086cdbd0ce87082ce77ae4b1651a35,mazed-jato,bdsaglam/drop-musique-mini,default,train,noop,no,bdsaglam/drop-musique-mini,default,validation,golden,1,qwen-2.5-32b,0.5,cot,1,0.02,0.492792,0.02,0.492792
7,ea162084da1636fb65a219eca1647b56f51a4dd7,inter-pupa,bdsaglam/drop-musique-mini,default,train,bfsrs-light,no,bdsaglam/drop-musique-mini,default,validation,golden,1,qwen-2.5-32b,0.5,cot,1,0.01,0.519027,0.01,0.519027


In [7]:
def sorted_tuple(x):
    return tuple(sorted(x))

run_counts = (
    df.groupby(param_cols[:-1])[
        "params.run"
    ]
    .aggregate(sorted_tuple)
    .reset_index()
)

In [8]:
for col in run_counts.columns:
    print(f"- {col}")
    for value in run_counts[col].unique():
        print(f"\t{value}")

- params.train.dataset.path
	bdsaglam/drop-musique-mini
- params.train.dataset.name
	default
- params.train.dataset.split
	train
- params.train.optimizer
	bfsrs-light
	miprov2-light
	noop
- params.train.ensemble
	no
- params.evaluation.dataset.path
	bdsaglam/drop-musique-mini
- params.evaluation.dataset.name
	default
- params.evaluation.dataset.split
	validation
- params.retrieve.model_type
	golden
- params.retrieve.top_k
	1
- params.qa.model
	qwen-2.5-32b
- params.qa.temperature
	0.0
	0.5
- params.qa.technique
	cot
- params.run
	(1,)


In [9]:
df.to_json('exps.jsonl', orient='records', lines=True)

In [11]:
df[["name", "params.qa.technique", "params.train.optimizer", "metrics.exact_match", "metrics.f1"]].sort_values(by="metrics.f1", ascending=False)

,name,params.qa.technique,params.train.optimizer,metrics.exact_match,metrics.f1
1,chief-aces,cot,miprov2-light,0.02,0.538309
3,worse-rasp,cot,bfsrs-light,0.02,0.531318
7,inter-pupa,cot,bfsrs-light,0.01,0.519027
4,irate-matt,cot,miprov2-light,0.01,0.517477
5,mazed-jato,cot,noop,0.02,0.492792
2,wired-prob,cot,noop,0.02,0.491522
